<h1> 📘 Biofilter — Reports to ETL Management </h1>

Biofilter provides a set of built-in ETL reports designed to help users monitor, inspect, and debug data ingestion pipelines. These reports expose the execution status, history, and metadata of ETL processes in a structured and reproducible way, making it easier to understand what has been loaded, what is currently running, and where failures may have occurred.

Methods

* report.run() — Execute a report and return the results as a pandas DataFrame.

Available Reports

* etl_status — High-level, consolidated view of ETL pipelines, showing the latest successful execution per data source and overall pipeline health.

* etl_packages — Detailed, row-level view of all ETL packages, exposing extract, transform, and load steps, timestamps, statuses, and logs for debugging and auditing.

--------

### Start Biofilter3R

In [1]:
from biofilter import Biofilter
bf = Biofilter()

[INFO] ════════════════════════════════════
[INFO] 🚀 Initializing Biofilter3R
[INFO]    • Version: 3.2.0
[INFO]    • Debug mode: False
[INFO]    • Config: /home/bioadmin/biofilter/.biofilter.toml
[INFO] ════════════════════════════════════
[INFO] 🔌 Database connection established
[INFO]    • Engine: postgresql+psycopg2
[INFO]    • Host:   localhost
[INFO]    • DB:     biofilter
[INFO]    • Time:   0.8 ms
[INFO] ════════════════════════════════════


------

### 1. Report to extract ETL Status in the Database

Returns a consolidated ETL status view per data source, showing the latest successful pipeline state (Extract → Transform → Load) for each selected data source.

This report is designed to answer the question:

* For these data sources, do we have a valid and up-to-date ETL pipeline?

In [2]:
bf.report.explain("etl_status")

📦 ETL Status (Latest Good)

This report summarizes ETL execution health per DataSource by selecting:
- The most recent GOOD extract package (completed or up-to-date)
- The most recent GOOD transform package
- The most recent GOOD load package

If the latest extract is newer but transform/load are missing or not aligned
(by hash), the report still shows the last good transform/load and flags them
as stale (not aligned with latest extract).



In [3]:
# df = bf.report.run("report_etl_status")
df = bf.report.run(
    "etl_status",
    source_system="ncbi",
    data_sources=["dbsnp_chr1", "dbsnp_chr2"],
)
df

,source_system,data_source,data_type,pipeline_ok,extract_package_id,extract_status,extract_end,transform_package_id,transform_status,transform_end,transform_aligned_with_latest_extract,load_package_id,load_status,load_end,load_aligned_with_latest_transform,latest_error
0,NCBI,dbsnp_chr1,Variant,True,40,completed,2025-12-12 02:48:36.670498,44,completed,2025-12-13 02:06:20.737723,True,70,completed,2025-12-13 16:39:57.230788,True,None
1,NCBI,dbsnp_chr2,Variant,True,45,completed,2025-12-12 05:50:29.425576,46,completed,2025-12-13 05:24:50.411411,True,74,completed,2025-12-14 00:27:13.428307,True,None


-------

### 2. Report to extract ETL the Packages history

The etl_packages report provides a detailed, row-level view of ETL execution history, returning one row per ETLPackage created in the system. It exposes full metadata for each package, including source system, data source, operation type (extract, transform, load), execution timestamps, per-stage statuses, hashes, row counts, and optional logs or error messages. This report is intended for debugging, auditing, and operational analysis, allowing users to inspect failed, running, skipped, or completed ETL steps, trace pipeline execution order, and understand exactly what happened during each ETL phase. Unlike etl_status, it does not consolidate results and instead reflects the raw ETL lifecycle as recorded in the database.

In [4]:
bf.report.explain("etl_packages")

📦 ETL Packages – Detailed Audit Report

This report provides a **raw, non-aggregated** view of the ETL execution state.

Each row corresponds to **one ETLPackage record**, which may represent:
- one ETL stage (extract / transform / load), or
- one full execution attempt, depending on how the ETL was triggered.

The report joins:
- ETLSourceSystem (e.g. NCBI, Ensembl, UniProt)
- ETLDataSource (e.g. dbSNP_chr1, ensembl, hgnc)
- ETLPackage (execution metadata)

This report is intentionally *not consolidated*.
It is designed for:
- Debugging failed or stuck jobs
- Auditing execution history
- Understanding how many packages were created per data source
- Verifying status transitions across ETL stages

Recommended usage:
- Use this report to identify inconsistencies
- Fix ETL status logic
- Only then create consolidated / dashboard-style reports



In [5]:
# df = bf.report.run("etl_packages")
df = bf.report.run(
    "etl_packages",
    source_system="ncbi",
    data_sources=["dbsnp_chr1"],
)
df.head()

,package_id,created_at,source_system,data_source,status,operation_type,version_tag,note,log,extract_status,...,transform_rows,transform_hash,load_status,load_start,load_end,load_rows,load_hash,extract_minutes,transform_minutes,load_minutes
0,70,2025-12-13 02:06:20.759949,NCBI,dbsnp_chr1,completed,load,None,None,None,pending,...,None,None,completed,2025-12-13 02:06:20.776466,2025-12-13 16:39:57.230788,None,cbddf8bbb56e1a8ee106c320fce7468b,NaN,NaN,873.607572
1,44,2025-12-12 02:48:36.734476,NCBI,dbsnp_chr1,completed,transform,None,None,None,pending,...,None,cbddf8bbb56e1a8ee106c320fce7468b,pending,NaT,NaT,None,None,NaN,1397.732143,NaN
2,40,2025-12-12 01:03:26.827303,NCBI,dbsnp_chr1,completed,extract,None,None,None,completed,...,None,None,pending,NaT,NaT,None,None,105.159639,NaN,NaN
